In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report


In [2]:
import shutil
import random

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Flatten, Input, BatchNormalization, Dense, Activation, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam

2024-12-04 13:45:21.098703: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-04 13:45:22.222057: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder

from copy import deepcopy

In [ ]:
# # Define paths
# dataset_path = 'BrainTumor_MRI/Training'
# class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']  # Assuming the subfolders are named accordingly

# # Parameters
# IMG_SIZE = 256


# # Plot the training and testing metrics
# plot_acc_loss("FedProx Training and Testing Metrics for mu=1.0",
#               train_loss_hist_4, train_acc_hist_4, test_loss_hist_4, test_acc_hist_4)

# # File to save the results
# output_file = "results_mu=1.0.txt"
# # Save the history to a file
# save_history_to_file(output_file, n_iter, train_loss_hist_4, train_acc_hist_4, test_loss_hist_4, test_acc_hist_4)

# https://chatgpt.com/share/670f84c9-a108-8000-a415-7248537697dc  - to get explanation related to file format

In [ ]:
import os
import shutil

# Define paths for training and testing datasets
source_train_dir = "BrainTumor_MRI/Training"
source_test_dir = "BrainTumor_MRI/Testing"

# Define class names
classes = ["glioma", "meningioma", "notumor", "pituitary"]

# Check if images are in the correct source directory
for class_name in classes:
    class_train_folder = os.path.join(source_train_dir, class_name)
    class_test_folder = os.path.join(source_test_dir, class_name)

    if not os.path.exists(class_train_folder):
        print(f"Source training folder for {class_name} does not exist!")
    if not os.path.exists(class_test_folder):
        print(f"Source testing folder for {class_name} does not exist!")
    else:
        print(f"Source folders exist and contain {len(os.listdir(class_train_folder))} training images and {len(os.listdir(class_test_folder))} testing images for {class_name}.")


In [ ]:
# ---------------- CBAM Block Definition ------------------
def cbam_block(inputs, reduction_ratio=0.5):
    channels = inputs.shape[-1]

    # Channel attention
    avg_pool = layers.GlobalAveragePooling2D()(inputs)
    max_pool = layers.GlobalMaxPooling2D()(inputs)
    shared_layer_1 = layers.Dense(int(channels * reduction_ratio), activation='relu', use_bias=True)
    shared_layer_2 = layers.Dense(channels, activation='relu', use_bias=True)

    avg_pool = shared_layer_1(avg_pool)
    avg_pool = shared_layer_2(avg_pool)
    max_pool = shared_layer_1(max_pool)
    max_pool = shared_layer_2(max_pool)

    attention = layers.Add()([avg_pool, max_pool])
    attention = layers.Activation('sigmoid')(attention)
    attention = layers.Reshape((1, 1, channels))(attention)
    scaled_inputs = layers.Multiply()([inputs, attention])

    # Spatial attention
    squeeze = layers.Conv2D(filters=1, kernel_size=1, activation='sigmoid', use_bias=False)(scaled_inputs)
    expanded_inputs = layers.Multiply()([scaled_inputs, squeeze])

    return expanded_inputs

# ---------------- Model Definitions ------------------

# ResNet50 with CBAM and a more complex output structure
def ResNet50_CBAM_Model():
    input_shape = (224, 224, 3)
    resnet50 = tf.keras.applications.ResNet50(include_top=False, input_shape=input_shape, weights='imagenet')

    x = layers.GlobalAveragePooling2D()(resnet50.output)
    x = layers.Dense(units=1024, activation='relu')(x)
    x = layers.Dense(units=512, activation='relu')(x)
    x = layers.Dense(units=256, activation='relu')(x)
    x = layers.Dense(units=128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Reshape((1, 1, 128))(x)
    x_max = layers.GlobalMaxPooling2D()(x)
    x_avg = layers.GlobalAveragePooling2D()(x)
    x = layers.Concatenate()([x_max, x_avg])
    x = layers.Dense(units=4, activation='softmax')(x)

    model = models.Model(inputs=resnet50.input, outputs=x, name='ResNet50_CBAM_Model')
    return model


In [ ]:
model = ResNet50_CBAM_Model()

# Compile models
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Paths to the dataset
train_dir = 'BrainTumor_MRI/Training'
test_dir = 'BrainTumor_MRI/Testing'
output_dir = 'NON_IID/'

# Class names
classes = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Number of samples to be taken for each model
samples = {
    'Model_1': {'glioma': (1021, 210), 'meningioma': (100, 32), 'notumor': (100, 50), 'pituitary': (100, 50)},
    'Model_2': {'glioma': (100, 30), 'meningioma': (1039, 210), 'notumor': (100, 50), 'pituitary': (100, 50)},
    'Model_3': {'glioma': (100, 30), 'meningioma': (100, 32), 'notumor': (1295, 255), 'pituitary': (100, 50)},
    'Model_4': {'glioma': (100, 30), 'meningioma': (100, 32), 'notumor': (100, 50), 'pituitary': (1157, 150)}
}

# Function to copy files to the appropriate directories
def copy_files(file_list, dest_dir):
    for file_path in file_list:
        shutil.copy(file_path, dest_dir)

# Splitting the dataset
for model, splits in samples.items():
    model_dir = os.path.join(output_dir, model)
    os.makedirs(model_dir, exist_ok=True)

    for cls, (train_size, test_size) in splits.items():
        # Train directory and files
        train_class_dir = os.path.join(train_dir, cls)
        train_files = [os.path.join(train_class_dir, f) for f in os.listdir(train_class_dir)]

        # Test directory and files
        test_class_dir = os.path.join(test_dir, cls)
        test_files = [os.path.join(test_class_dir, f) for f in os.listdir(test_class_dir)]

        # Split the train files for the model
        train_subset, _ = train_test_split(train_files, train_size=train_size, random_state=42)

        # Split the test files for the model
        test_subset, _ = train_test_split(test_files, train_size=test_size, random_state=42)

        # Copy train files
        model_train_dir = os.path.join(model_dir, 'train', cls)
        os.makedirs(model_train_dir, exist_ok=True)
        copy_files(train_subset, model_train_dir)

        # Copy test files
        model_test_dir = os.path.join(model_dir, 'test', cls)
        os.makedirs(model_test_dir, exist_ok=True)
        copy_files(test_subset, model_test_dir)

print("Dataset split completed successfully.")


Dataset split completed successfully.


In [ ]:
import os
import shutil

# Define paths for training and testing datasets
source_train_dir = "Federated_Learning_NON_IID/Model_1/train"
source_test_dir = "Federated_Learning_NON_IID/Model_1/test"

# Define class names
classes = ["glioma", "meningioma", "notumor", "pituitary"]

# Check if images are in the correct source directory
for class_name in classes:
    class_train_folder = os.path.join(source_train_dir, class_name)
    class_test_folder = os.path.join(source_test_dir, class_name)

    if not os.path.exists(class_train_folder):
        print(f"Source training folder for {class_name} does not exist!")
    if not os.path.exists(class_test_folder):
        print(f"Source testing folder for {class_name} does not exist!")
    else:
        print(f"Source folders exist and contain {len(os.listdir(class_train_folder))} training images and {len(os.listdir(class_test_folder))} testing images for {class_name}.")


In [ ]:
def loss_classifier(predictions, labels):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=predictions)
    return tf.reduce_mean(loss)


def loss_dataset(model, dataset, loss_f):
    loss = 0
    for idx, (features, labels) in enumerate(dataset):
        # Convert PyTorch tensors to NumPy and then to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        predictions = model(features_tf)
        loss += loss_f(predictions, labels_tf)
    loss /= (idx + 1)
    return loss


def accuracy_dataset(model, dataset):
    correct = 0
    total = 0
    for features, labels in dataset:
        # Convert PyTorch tensors to NumPy and then to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        predictions = model(features_tf)
        predicted = tf.argmax(predictions, axis=1, output_type=tf.int32)
        correct += tf.reduce_sum(tf.cast(tf.equal(predicted, labels_tf), tf.int32)).numpy()
        total += labels_tf.shape[0]

    accuracy = 100 * correct / total
    return accuracy


def train_step(model, model_0, mu, optimizer, train_data, loss_f):
    total_loss = 0
    for idx, (features, labels) in enumerate(train_data):
        # Convert PyTorch tensors to NumPy and then to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        with tf.GradientTape() as tape:
            predictions = model(features_tf)
            loss = loss_f(predictions, labels_tf)
            loss += mu / 2 * difference_models_norm_2(model, model_0)

        total_loss += loss
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return total_loss / (idx + 1)


def local_learning(model, mu, optimizer, train_data, epochs, loss_f):
    # Clone the model instead of using deepcopy
    model_0 = tf.keras.models.clone_model(model)
    model_0.set_weights(model.get_weights())  # Set weights to be identical initially

    for e in range(epochs):
        local_loss = train_step(model, model_0, mu, optimizer, train_data, loss_f)

    return local_loss


def difference_models_norm_2(model_1, model_2):
    norm = tf.reduce_sum([tf.reduce_sum(tf.square(w1 - w2)) for w1, w2 in zip(model_1.trainable_variables, model_2.trainable_variables)])
    return norm

def set_to_zero_model_weights(model):
    for layer_weights in model.trainable_variables:
        layer_weights.assign(tf.zeros_like(layer_weights))

def average_models(model, clients_models_hist, weights):
    set_to_zero_model_weights(model)
    for k, client_hist in enumerate(clients_models_hist):
        for idx, layer_weights in enumerate(model.trainable_variables):
            contribution = client_hist[idx] * weights[k]
            layer_weights.assign_add(contribution)

#_______________________________________________________

In [ ]:
def FedProx(model, training_sets, n_iter, testing_sets, mu=0, epochs=5, lr=0.01, decay=1):
    # Verify that `model` is a Keras model instance
    if not isinstance(model, tf.keras.Model):
        raise TypeError("The provided model is not a TensorFlow Keras model. Please provide a valid Keras model.")

    loss_f = loss_classifier
    K = len(training_sets)
    n_samples = sum([len(db) for db in training_sets])
    weights = [len(db) / n_samples for db in training_sets]
    print("Clients' weights:", weights)

    # Initialize history lists for training and testing
    train_loss_hist = []
    train_acc_hist = []
    test_loss_hist = []
    test_acc_hist = []
    models_hist = []

    for i in range(n_iter):
        clients_params = []
        clients_losses = []
        clients_accuracies = []

        for k in range(K):
            # Clone the model and set weights for local training
            local_model = tf.keras.models.clone_model(model)
            local_model.set_weights(model.get_weights())

            local_optimizer = tf.keras.optimizers.SGD(learning_rate=lr)

            # Perform local training and track the loss
            local_loss = local_learning(local_model, mu, local_optimizer, training_sets[k], epochs, loss_f)
            clients_losses.append(local_loss)

            # Track training accuracy for the client
            train_acc = accuracy_dataset(local_model, training_sets[k])
            clients_accuracies.append(train_acc)

            # Store model parameters (deep copy to ensure immutability)
            clients_params.append([tf.identity(tens_param) for tens_param in local_model.trainable_variables])

        # Average the local models into the global model
        average_models(model, clients_params, weights=weights)
        models_hist.append(deepcopy(clients_params))

        # Collect metrics for this iteration
        train_loss_hist.append(clients_losses)
        train_acc_hist.append(clients_accuracies)

        # Compute testing metrics using the global model
        test_loss_hist.append([loss_dataset(model, dl, loss_f).numpy() for dl in testing_sets])
        test_acc_hist.append([accuracy_dataset(model, dl) for dl in testing_sets])

        # Update learning rate by decay factor
        lr *= decay
        print(f'====> i: {i+1} Server Test Accuracy: {test_acc_hist[-1]}')

    return model, train_loss_hist, train_acc_hist, test_loss_hist, test_acc_hist



In [ ]:
import matplotlib.pyplot as plt

def plot_acc_loss(title: str, train_loss_hist: list, train_acc_hist: list,
                  test_loss_hist: list, test_acc_hist: list):
    plt.figure(figsize=(15, 5))  # Make the plot wider

    # Plot training loss
    plt.subplot(2, 2, 1)
    lines = plt.plot(train_loss_hist)
    plt.title("Training Loss")
    plt.legend(lines, [f"C{i+1}" for i in range(len(train_loss_hist[0]))])

    # Plot training accuracy
    plt.subplot(2, 2, 2)
    lines = plt.plot(train_acc_hist)
    plt.title("Training Accuracy")
    plt.legend(lines, [f"C{i+1}" for i in range(len(train_acc_hist[0]))])

    # Plot testing loss
    plt.subplot(2, 2, 3)
    lines = plt.plot(test_loss_hist)
    plt.title("Testing Loss")
    plt.legend(lines, [f"C{i+1}" for i in range(len(test_loss_hist[0]))])

    # Plot testing accuracy
    plt.subplot(2, 2, 4)
    lines = plt.plot(test_acc_hist)
    plt.title("Testing Accuracy")
    plt.legend(lines, [f"C{i+1}" for i in range(len(test_acc_hist[0]))])

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()


def save_history_to_file(filename, n_iter, train_loss_hist, train_acc_hist,
                         test_loss_hist, test_acc_hist):
    with open(filename, 'w') as f:
        f.write("FedProx Training Results\n")
        f.write("="*50 + "\n")
        f.write(f"Number of iterations: {n_iter}\n\n")

        # Write the history for each iteration
        for i in range(n_iter):
            f.write(f"Iteration {i+1}:\n")
            f.write(f"Train Loss: {train_loss_hist[i]}\n")
            f.write(f"Train Accuracy: {train_acc_hist[i]}\n")
            f.write(f"Test Loss: {test_loss_hist[i]}\n")
            f.write(f"Test Accuracy: {test_acc_hist[i]}\n")
            f.write("-"*50 + "\n")

    print(f"Training history saved to {filename}")

# Data Agumentation
###'Model 1': {'glioma': (1021, 210), 'meningioma': (100, 32), 'notumor': (100, 50), 'pituitary': (100, 50)},
###'Model 2': {'glioma': (100, 30), 'meningioma': (1039, 210), 'notumor': (100, 50), 'pituitary': (100, 50)},
###'Model 3': {'glioma': (100, 30), 'meningioma': (100, 32), 'notumor': (1295, 255), 'pituitary': (100, 50)},
###'Model 4': {'glioma': (100, 30), 'meningioma': (100, 32), 'notumor': (100, 50), 'pituitary': (1157, 150)}

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data augmentation strategy
augmentation = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=90,
    width_shift_range=0.1,
    height_shift_range=0.1,
    channel_shift_range=0.1, # extra
    vertical_flip=True,
    horizontal_flip=True,
    fill_mode="nearest"
)


In [ ]:
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img, array_to_img
import numpy as np

def augment_class_images(input_dir, target_dir, target_count, augmentation):
    current_count = len(os.listdir(input_dir))
    n_to_generate = target_count - current_count

    # If current count is already sufficient, no augmentation is needed
    if n_to_generate <= 0:
        print(f"No augmentation needed for {input_dir}")
        return

    img_files = os.listdir(input_dir)
    i = 0

    while i < n_to_generate:
        img_file = img_files[i % current_count]
        img_path = os.path.join(input_dir, img_file)

        # Load and augment image
        image = load_img(img_path)
        image_array = img_to_array(image)
        image_array = np.expand_dims(image_array, axis=0)

        # Generate augmented images
        for batch in augmentation.flow(image_array, batch_size=1):
            new_img = array_to_img(batch[0], scale=True)
            new_img.save(os.path.join(target_dir, f"aug_{i}.jpg"))
            i += 1
            if i >= n_to_generate:
                break


Training

In [ ]:

augment_class_images(
    input_dir="Federated_Learning_NON_IID/Model_4/train/glioma",
    target_dir="Federated_Learning_NON_IID/Model_4/train/glioma",
    target_count=1157,
    augmentation=augmentation
)

augment_class_images(
    input_dir="Federated_Learning_NON_IID/Model_4/train/meningioma",
    target_dir="Federated_Learning_NON_IID/Model_4/train/meningioma",
    target_count=1157,
    augmentation=augmentation
)

augment_class_images(
    input_dir="Federated_Learning_NON_IID/Model_4/train/notumor",
    target_dir="Federated_Learning_NON_IID/Model_4/train/notumor",
    target_count=1157,
    augmentation=augmentation
)

Testing data agumentation

In [ ]:

augment_class_images(
    input_dir="Federated_Learning_NON_IID/Model_4/test/glioma",
    target_dir="Federated_Learning_NON_IID/Model_4/test/glioma",
    target_count=150,
    augmentation=augmentation
)

augment_class_images(
    input_dir="Federated_Learning_NON_IID/Model_4/test/meningioma",
    target_dir="Federated_Learning_NON_IID/Model_4/test/meningioma",
    target_count=150,
    augmentation=augmentation
)

augment_class_images(
    input_dir="Federated_Learning_NON_IID/Model_4/test/notumor",
    target_dir="Federated_Learning_NON_IID/Model_4/test/notumor",
    target_count=150,
    augmentation=augmentation
)

In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Directories for each client
split_dirs = [
    "Federated_Learning_NON_IID/Model_1",
    "Federated_Learning_NON_IID/Model_2",
    "Federated_Learning_NON_IID/Model_3",
    "Federated_Learning_NON_IID/Model_4"
]

# Number of clients
n_clients = len(split_dirs)

# Updated image transformations: Normalize first, then permute
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert image to tensor format (C, H, W)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    transforms.Lambda(lambda x: x.permute(1, 2, 0))  # Permute to (H, W, C)
])

def get_tumor_dataloaders(split_dirs, batch_size=25, shuffle=True):
    """
    Returns data loaders for all clients for both training and testing sets.
    """
    tumor_iid_train_dls = []
    tumor_iid_test_dls = []

    for client_idx, client_dir in enumerate(split_dirs):
        # Get the directory for the current client
        train_dir = os.path.join(client_dir, 'train')
        test_dir = os.path.join(client_dir, 'test')

        # Check if the directories exist
        if not os.path.exists(train_dir) or not os.path.exists(test_dir):
            print(f"Directory not found for client {client_idx + 1}:")
            print(f"Train dir: {train_dir}")
            print(f"Test dir: {test_dir}")
            continue

        # Load training data for the current client
        train_dataset = datasets.ImageFolder(train_dir, transform=transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

        # Load testing data for the current client
        test_dataset = datasets.ImageFolder(test_dir, transform=transform)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Append the dataloaders for the current client to the list
        tumor_iid_train_dls.append(train_loader)
        tumor_iid_test_dls.append(test_loader)

    return tumor_iid_train_dls, tumor_iid_test_dls

# Get the training and testing data loaders
Tumor_iid_train_dls, Tumor_iid_test_dls = get_tumor_dataloaders(split_dirs, batch_size=25)

# Checking the sizes of the images in the data loaders to verify the shape
for batch_idx, (images, labels) in enumerate(Tumor_iid_train_dls[0]):  # Checking for client 1
    print(f"Batch {batch_idx} image sizes: {images.size()}")  # Should print torch.Size([25, 224, 224, 3])
    break  # Check only the first batch


In [ ]:
n_iter = 21
# Execute FedProx with the new 4-client setup
model, train_loss, train_acc, test_loss, test_acc = FedProx(
    model, Tumor_iid_train_dls, n_iter, Tumor_iid_test_dls, mu=0.3, epochs=1, lr=0.001
)

In [ ]:
n_iter = 20
# Execute FedProx with the new 4-client setup
model_1, train_loss_1, train_acc_1, test_loss_1, test_acc_1 = FedProx(
    model, Tumor_iid_train_dls, n_iter, Tumor_iid_test_dls, mu=0.4, epochs=1, lr=0.001
)

In [ ]:
# File to save the results
output_file = "results-mu=0.4.txt"
# Save the history to a file
save_history_to_file(output_file, n_iter, train_loss_1, train_acc_1, test_loss_1, test_acc_1)


In [ ]:
# Plot the training and testing metrics
plot_acc_loss("FedProx Training and Testing Metrics for mu=0.4",
              train_loss_1, train_acc_1, test_loss_1, test_acc_1)

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import tensorflow as tf
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report
import numpy as np
import os

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert image to tensor format (C, H, W)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    transforms.Lambda(lambda x: x.permute(1, 2, 0))  # Permute to (H, W, C)
])

# Load the testing dataset
test_dir = 'BrainTumor_MRI/Testing'
test_dataset = datasets.ImageFolder(test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Class names mapping (adjust if necessary)
class_names = test_dataset.classes



In [ ]:
# Function to evaluate the global model on the testing dataset
def evaluate_global_model(model, test_loader):
    all_preds = []
    all_labels = []
    total_loss = 0

    for features, labels in test_loader:
        # Convert to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        # Get predictions
        predictions = model(features_tf)
        predicted = tf.argmax(predictions, axis=1, output_type=tf.int32)

        # Accumulate predictions and labels
        all_preds.extend(predicted.numpy())
        all_labels.extend(labels_np)

        # Compute loss
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_tf, logits=predictions)
        total_loss += tf.reduce_mean(loss).numpy()

    # Compute metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_labels, all_preds)

    print(f"\nConfusion Matrix:\n{conf_matrix}")
    print(f"\nAccuracy: {accuracy * 100:.2f}%")
    print(f"Loss: {total_loss / len(test_loader):.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

    # Print detailed classification report
    print("\nClassification Report:\n", classification_report(all_labels, all_preds, target_names=class_names))



In [ ]:
# Evaluate the model on the testing dataset
evaluate_global_model(model_1, test_loader)

In [ ]:
import tensorflow as tf

# Define the path to save the Keras model
MODEL_PATH = 'global_model_keras/'

# Save the trained global model in the Keras HDF5 or SavedModel format
model_1.save(MODEL_PATH)  # This ensures the model includes Keras metadata

print(f"Global model saved successfully at: {MODEL_PATH}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg19 import preprocess_input
import numpy as np

# Load the saved Keras model
MODEL_PATH = 'global_model_keras/'
model = tf.keras.models.load_model(MODEL_PATH)

# Define class names
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Function to predict the class of a single image
def predict_image(image_path, model):
    # Load and preprocess the image
    image = load_img(image_path, target_size=(224, 224))  # Resize to 224x224
    image_array = img_to_array(image)  # Convert to numpy array
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    image_array = preprocess_input(image_array)  # Normalize for VGG19

    # Perform inference
    predictions = model(image_array, training=False)  # Inference mode
    probabilities = tf.nn.softmax(predictions[0]).numpy()  # Apply softmax
    predicted_class = np.argmax(probabilities)

    # Print the predicted class and probabilities
    print(f"Predicted Class: {class_names[predicted_class]}")
    print(f"Probabilities: {probabilities}")

# Test the model with a single image
image_path = 'BrainTumor_MRI/M.jpg'
predict_image(image_path, model_1)


In [ ]:
image_path = 'BrainTumor_MRI/P.jpg'

In [ ]:
predict_image(image_path, model_1)

In [ ]:
MODEL_PATH = 'global_model_keras/'
model = tf.keras.models.load_model(MODEL_PATH)

In [ ]:
n_iter = 30
# Execute FedProx with the new 4-client setup
model_f, train_loss_f, train_acc_f, test_loss_f, test_acc_f = FedProx(
    model, Tumor_iid_train_dls, n_iter, Tumor_iid_test_dls, mu=0.4, epochs=1, lr=0.001
)

In [ ]:
import tensorflow as tf

# File to save the results
output_file = "results-mu=0.4_30-rounds.txt"

# Initial parameters
MODEL_PATH = 'global_model_keras/global_model_rounds_10.h5'
n_iter = 30
rounds_per_segment = 5
mu = 0.4
epochs = 1
lr = 0.001

# Initialize lists to store cumulative training and testing metrics
all_train_loss = []
all_train_acc = []
all_test_loss = []
all_test_acc = []

# Load the initial model
model = tf.keras.models.load_model(MODEL_PATH)

# Function for FedProx training in segments
for i in range(0, n_iter, rounds_per_segment):
    print(f"Starting rounds {i+1} to {i+rounds_per_segment}")
    
    # Train for the specified number of rounds in this segment
    model, train_loss_f, train_acc_f, test_loss_f, test_acc_f = FedProx(
        model, Tumor_iid_train_dls, rounds_per_segment, Tumor_iid_test_dls, mu=mu, epochs=epochs, lr=lr
    )
    
    # Append the metrics to the cumulative lists
    all_train_loss.extend(train_loss_f)
    all_train_acc.extend(train_acc_f)
    all_test_loss.extend(test_loss_f)
    all_test_acc.extend(test_acc_f)
    
    # Save the model at the end of each segment
    segment_model_path = f'{MODEL_PATH}global_model_rounds_{i+rounds_per_segment}.h5'
    model.save(segment_model_path)
    print(f"Model saved at: {segment_model_path}")

    # Reload the model to ensure continuity for the next segment
    model = tf.keras.models.load_model(segment_model_path)

# Save the cumulative history to a file after all rounds are complete
save_history_to_file(output_file, n_iter, all_train_loss, all_train_acc, all_test_loss, all_test_acc)

# Plot the cumulative training and testing metrics
plot_acc_loss("FedProx Training and Testing Metrics for mu=0.4, 30 rounds",
              all_train_loss, all_train_acc, all_test_loss, all_test_acc)

print("Training complete and results saved.")


In [ ]:
import tensorflow as tf

# File to save the results
output_file = "results-mu=0.4_30-rounds.txt"

# Initial parameters
MODEL_PATH = 'global_model_keras/global_model_rounds_15.h5'
n_iter = 30
rounds_per_segment = 5
mu = 0.4
epochs = 1
lr = 0.001

# Initialize lists to store cumulative training and testing metrics
all_train_loss = []
all_train_acc = []
all_test_loss = []
all_test_acc = []

# Load the initial model
model = tf.keras.models.load_model(MODEL_PATH)

# Function for FedProx training in segments
for i in range(15, n_iter, rounds_per_segment):
    print(f"Starting rounds {i+1} to {i+rounds_per_segment}")
    
    # Train for the specified number of rounds in this segment
    model, train_loss_f, train_acc_f, test_loss_f, test_acc_f = FedProx(
        model, Tumor_iid_train_dls, rounds_per_segment, Tumor_iid_test_dls, mu=mu, epochs=epochs, lr=lr
    )
    
    # Append the metrics to the cumulative lists
    all_train_loss.extend(train_loss_f)
    all_train_acc.extend(train_acc_f)
    all_test_loss.extend(test_loss_f)
    all_test_acc.extend(test_acc_f)
    
    # Save the model at the end of each segment
    segment_model_path = f'{MODEL_PATH}global_model_rounds_{i+rounds_per_segment}.h5'
    model.save(segment_model_path)
    print(f"Model saved at: {segment_model_path}")

    # Reload the model to ensure continuity for the next segment
    model = tf.keras.models.load_model(segment_model_path)

# Save the cumulative history to a file after all rounds are complete
save_history_to_file(output_file, n_iter, all_train_loss, all_train_acc, all_test_loss, all_test_acc)

# Plot the cumulative training and testing metrics
plot_acc_loss("FedProx Training and Testing Metrics for mu=0.4, 30 rounds",
              all_train_loss, all_train_acc, all_test_loss, all_test_acc)

print("Training complete and results saved.")


In [ ]:
# Save the cumulative history to a file after all rounds are complete
# Only use data for the last 15 rounds
last_15_train_loss = all_train_loss[-15:]  # Get last 15 entries
last_15_train_acc = all_train_acc[-15:]
last_15_test_loss = all_test_loss[-15:]
last_15_test_acc = all_test_acc[-15:]

# Save the last 15 rounds of history to a file
save_history_to_file(output_file, 15, last_15_train_loss, last_15_train_acc, last_15_test_loss, last_15_test_acc)

# Plot the training and testing metrics for the last 15 rounds
plot_acc_loss("FedProx Training and Testing Metrics (Last 15 Rounds, mu=0.4)", 
              last_15_train_loss, last_15_train_acc, last_15_test_loss, last_15_test_acc)


In [ ]:
# Extract data for the last 15 rounds
last_15_train_loss = all_train_loss[-15:]  # Get last 15 entries
last_15_train_acc = all_train_acc[-15:]
last_15_test_loss = all_test_loss[-15:]
last_15_test_acc = all_test_acc[-15:]

# Define the output file path
output_file = "last_15_rounds_history.txt"

# Save the last 15 rounds of history to a file
def save_last_15_history_to_file(filename, train_loss, train_acc, test_loss, test_acc):
    with open(filename, "w") as f:
        f.write("FedProx Training and Testing Metrics (Last 15 Rounds)\n")
        f.write("=" * 50 + "\n")
        
        for i in range(len(train_loss)):
            try:
                # Handle cases where elements may be lists
                train_loss_val = float(train_loss[i][0]) if isinstance(train_loss[i], list) else float(train_loss[i])
                train_acc_val = float(train_acc[i][0]) if isinstance(train_acc[i], list) else float(train_acc[i])
                test_loss_val = float(test_loss[i][0]) if isinstance(test_loss[i], list) else float(test_loss[i])
                test_acc_val = float(test_acc[i][0]) if isinstance(test_acc[i], list) else float(test_acc[i])
                
                f.write(f"Round {i + 16}:\n")  # Adjust round numbers to 16-30
                f.write(f"Train Loss: {train_loss_val:.4f}\n")
                f.write(f"Train Accuracy: {train_acc_val:.4f}\n")
                f.write(f"Test Loss: {test_loss_val:.4f}\n")
                f.write(f"Test Accuracy: {test_acc_val:.4f}\n")
                f.write("-" * 50 + "\n")
                
            except (ValueError, TypeError) as e:
                f.write(f"Error processing round {i + 16}: {e}\n")
                f.write("-" * 50 + "\n")
    
    print(f"History for the last 15 rounds saved to {filename}")

# Save the data to the file
save_last_15_history_to_file(output_file, last_15_train_loss, last_15_train_acc, last_15_test_loss, last_15_test_acc)

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import tensorflow as tf
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report
import numpy as np
import os

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert image to tensor format (C, H, W)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    transforms.Lambda(lambda x: x.permute(1, 2, 0))  # Permute to (H, W, C)
])

# Load the testing dataset
test_dir = 'BrainTumor_MRI/Testing'
test_dataset = datasets.ImageFolder(test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Class names mapping (adjust if necessary)
class_names = test_dataset.classes



In [ ]:
# Define class names
class_names = ["glioma", "meningioma", "notumor", "pituitary"]  # Modify according to your classes

# Function to evaluate the global model on the testing dataset
def evaluate_global_model(model, test_loader):
    all_preds = []
    all_labels = []
    total_loss = 0

    for features, labels in test_loader:
        # Convert to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        # Get predictions
        predictions = model(features_tf)
        predicted = tf.argmax(predictions, axis=1, output_type=tf.int32)

        # Accumulate predictions and labels
        all_preds.extend(predicted.numpy())
        all_labels.extend(labels_np)

        # Compute loss
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_tf, logits=predictions)
        total_loss += tf.reduce_mean(loss).numpy()

    # Compute metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_labels, all_preds)

    # Print detailed classification report
    print("\nClassification Report:\n", classification_report(all_labels, all_preds, target_names=class_names))
    print(f"\nAccuracy: {accuracy * 100:.2f}%")
    print(f"Loss: {total_loss / len(test_loader):.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

    # Plot confusion matrix using Seaborn
    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted3 Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()


In [ ]:
MODEL_PATH = 'global_model_keras/global_model_rounds_30.h5'
model = tf.keras.models.load_model(MODEL_PATH)

In [ ]:
# Evaluate the model on the testing dataset
evaluate_global_model(model, test_loader)

In [ ]:
MODEL_PATH = 'global_model_keras/global_model_rounds_25.h5'
model_a = tf.keras.models.load_model(MODEL_PATH)

In [ ]:
evaluate_global_model(model_a, test_loader)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg19 import preprocess_input
import numpy as np

# Define class names
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Function to predict the class of a single image
def predict_image(image_path, model):
    # Load and preprocess the image
    image = load_img(image_path, target_size=(224, 224))  # Resize to 224x224
    image_array = img_to_array(image)  # Convert to numpy array
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    image_array = preprocess_input(image_array)  # Normalize for VGG19

    # Perform inference
    predictions = model(image_array, training=False)  # Inference mode
    probabilities = tf.nn.softmax(predictions[0]).numpy()  # Apply softmax
    predicted_class = np.argmax(probabilities)

    # Print the predicted class and probabilities
    print(f"Predicted Class: {class_names[predicted_class]}")
    print(f"Probabilities: {probabilities}")


In [ ]:
# Test the model with a single image
image_path = 'BrainTumor_MRI/M.jpg'
predict_image(image_path, model)

In [ ]:
image_path = 'BrainTumor_MRI/Testing/pituitary/Te-pi_0024.jpg'
predict_image(image_path, model)

In [ ]:
image_path = 'BrainTumor_MRI/Testing/pituitary/Te-pi_0024.jpg'
predict_image(image_path, model)

In [ ]:
image_path = 'BrainTumor_MRI/Testing/meningioma/Te-me_0073.jpg'
predict_image(image_path, model)

In [ ]:
# Define a range of mu values to search
n_iter = 10
mu_values = [0.9, 1.0]
best_mu = None
best_test_loss = float('inf')  # Set to inf to minimize loss
best_test_acc = 0.0  # Set to 0 if you want to maximize accuracy

# Grid search over mu values
for mu in mu_values:
    # Execute FedProx with the current mu value
    model_test, train_loss, train_acc, test_loss, test_acc = FedProx(
        model, Tumor_iid_train_dls, n_iter, Tumor_iid_test_dls, mu=mu, epochs=1, lr=0.001
    )
    
    # Update best_mu if current mu yields better test performance
    if test_loss < best_test_loss:  # Adjust if optimizing accuracy
        best_mu = mu
        best_test_loss = test_loss
        best_test_acc = test_acc

print(f"Best mu: {best_mu} with Test Loss: {best_test_loss} and Test Accuracy: {best_test_acc}")
